In [1]:
# Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("data/chapter_2.txt")
document=loader.load()
document

[Document(metadata={'source': 'data/chapter_2.txt'}, page_content='Chapter 2: The Yoga of Knowledge (Sankhya Yoga)\n\nShloka 1:\nArjuna, overwhelmed with grief, refuses to fight his relatives in battle\nKey Concept: Moral dilemma and emotional turmoil\n\n \n\nShloka 2:\nKrishna chides Arjuna for his weakness and unmanliness\nKey Teaching: Importance of righteous duty over emotional attachments\n\n \n\nShloka 7:\nArjuna admits his confusion and seeks Krishna\'s guidance\nKey Theme: Student-teacher dynamic in spiritual learning\n\n \n\nShloka 11:\nThe eternal nature of the soul is explained\nCore Message: Soul cannot be destroyed - it\'s everlasting\n\n \n\nShloka 13:\nReincarnation explained through body changes\nAnalogy: Changing clothes doesn\'t change the wearer\n\n \n\nShloka 14:\nTemporary nature of sensory experiences\nTeaching: Endure pleasure/pain like passing seasons\n\n \n\nShloka 15:\nThe stable mind remains undisturbed\nKey Quality: Equanimity in success/failure\n\n \n\nShlo

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
textSplitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
)
document = textSplitter.split_documents(document)

In [3]:
# Embedding & Vector Store Creation
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(
    documents=document, embedding=OllamaEmbeddings(model="mistral")
)

C:\Users\adity\AppData\Local\Temp\ipykernel_10924\4224101563.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  documents=document, embedding=OllamaEmbeddings(model="mistral")


In [4]:
db

In [5]:
# Vector db  query
query="What is the main teaching of chapter 2 of geeta?"
result = db.similarity_search(query)
result[0].page_content

"Shloka 27:\nInevitable nature of death\nExistential Truth: What's born must die\n\n \n\nShloka 30:\nSoul is eternal and transcendental\nSpiritual Insight: Witness consciousness\n\n \n\nShloka 31:\nDuty of a warrior to fight righteous war\nEthical Framework: Dharma in action\n\n \n\nShloka 33:\nWarning against abandoning duty\nConsequence: Loss of honor and purpose\n\n \n\nShloka 38:\nEquanimity in victory and defeat\nMental Discipline: Focus on action, not results"

In [6]:
# Set up LLM
from langchain_community.llms import Ollama
llm = Ollama(model="mistral")
llm

C:\Users\adity\AppData\Local\Temp\ipykernel_10924\863618470.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


Ollama(model='mistral')

In [7]:
# Prompting
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """Answer the question based on the context: {context}
    Question: {input}"""
)

In [8]:
# Chain Creation
# llm -- prompt

from langchain.chains.combine_documents import create_stuff_documents_chain
chain=create_stuff_documents_chain(llm=llm, prompt=prompt)

In [9]:
# Retriever Creation
# Retriever -- Vector Store

retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024358C87250>, search_kwargs={})

In [10]:
# Retriever Chain Creation
"""
       Vector Store
            |
User -> Retriever -> LLM -- Prompt
"""
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(
    retriever, chain
)

In [14]:
response = retrieval_chain.invoke({"input":"Summarize Krishna’s 3 key arguments to convince Arjuna to fight"})

In [15]:
response['answer']

" Krishna's three key arguments to convince Arjuna to fight, as expressed in the Bhagavad Gita, are as follows:\n\n1. The Soul is Eternal and Indestructible (Shlokas 11, 20, 30): Krishna explains that the soul is eternal, birthless, deathless, and transcendental. It cannot be destroyed or harmed. This spiritual insight serves as a reminder that the actions of war are happening on a physical level, while the soul remains unaffected by such temporary events.\n\n2. Duty and Honor (Shlokas 31, 33): Krishna emphasizes that it is a warrior's duty to fight in righteous wars and uphold dharma (ethics). Abandoning this duty results in the loss of honor, purpose, and self-respect.\n\n3. Equanimity and Mental Discipline (Shlokas 15, 38): Krishna teaches Arjuna to maintain equanimity in victory and defeat by focusing on action rather than results. He urges him to endure pleasure and pain like passing seasons, while remaining detached and unattached to worldly desires and outcomes. By doing so, one